In [1]:
import joblib
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.colors as colors
from matplotlib_venn import venn2
import statistics 
import random
import csv

%matplotlib inline

In [2]:
pd.options.mode.chained_assignment = None 

In [3]:
HERE = os.path.dirname(os.path.abspath('__file__'))
ROOT = os.path.abspath(os.path.join(HERE, os.pardir))
DATA = os.path.join(ROOT, 'data')

SSGSEA_PRAD_KEGG = os.path.join(
    DATA,
    "kegg_prad.tsv"
)

TRAINED_MODEL = os.path.join(
    DATA,
    'trained_models',
    "prad_trained_model.joblib"
)

PRAD_LABELS = os.path.join(
    DATA,
    "phenotype_classes_prad.cls"
)

DRUGCENTRAL_TO_GENES_ID = os.path.join(
    DATA,
    "drugcentral_mapped_to_drugbank.tsv"
)

ALL_HGNC_GENES = os.path.join(
    DATA,
    "hgnc_symbols.csv"
)

GENE_SETS_KEGG = os.path.join(
    DATA,
    "kegg_geneset_final.gmt"
)

In [4]:
## Lable preparation

prad_labels = pd.read_csv(PRAD_LABELS, sep = "\t")
prad_labels.drop(prad_labels.index[0], inplace=True)
prad_labels = prad_labels.rename(index={1:'label'})
prad_labels = prad_labels.transpose()

temp_lable = []

for lable in prad_labels.label:
    temp_lable = lable.split(' ')

In [5]:
## Patient_Pathway dataframe preparation

# Transpose the dataframe's columns and rows
raw_data = pd.read_csv(SSGSEA_PRAD_KEGG, sep = "\t", index_col=0).transpose()

# Append the data lable as a column to main dataframe
raw_data.insert(311, "label", temp_lable, True)

# Convert the data lable into numerical value
num_labels = {"Normal": 0, "Tumor": 1} 
raw_data.label = [num_labels[item] for item in raw_data.label]

lable_list = list(raw_data.label.values)


# # Removing the index column 
raw_data.reset_index(drop=True, inplace=True)

In [6]:
def parse_gmt_file(gmt_path: str, min_size=3, max_size=3000):
    """Parse gmt file of pathways and their corresponding genes."""
    with open(gmt_path) as f:
        genesets_dict = {
            line.strip().split("\t")[0]: line.strip().split("\t")[2:]
            for line in f
        }
    return genesets_dict

In [7]:
pathway_genes_dict = parse_gmt_file(GENE_SETS_KEGG)

In [8]:
#HGNC_ID_map_to_gene_name = pd.read_csv(HGNC_ID_MAP_TO_GENE_NAME, sep = "\t")
all_hgnc_genes = pd.read_csv(ALL_HGNC_GENES, sep = "\t")

In [9]:
## Read the drugbank_to_genes file and filter all source_databases but drugbank

drugbank_to_genes_ID = pd.read_csv('https://raw.githubusercontent.com/drug2ways/results/master/networks/data/custom_network.tsv',sep = "\t")

In [10]:
## Filter all source_databases but drugbank
drugbank_to_genes_ID_keep_drugbank = drugbank_to_genes_ID.loc[drugbank_to_genes_ID['source_database'] == "drugbank"]

for i in range(len(drugbank_to_genes_ID_keep_drugbank["source"])):
    temp_drug = drugbank_to_genes_ID_keep_drugbank["source"].iloc[i]
    temp_drug = temp_drug.split(':')[1]
    drugbank_to_genes_ID_keep_drugbank["source"].iloc[i] = temp_drug

In [11]:
drugcentral = pd.read_csv(DRUGCENTRAL_TO_GENES_ID,sep = "\t")

columnsTitles = ['drugbank_id', 'hgnc_symbol', 'relation']

drugcentral = drugcentral.reindex(columns=columnsTitles)
drugcentral.rename(columns={'drugbank_id': 'source', 'hgnc_symbol': 'target', 'relation': 'relation' }, inplace=True)

In [12]:
#drugbank = drugbank_to_genes_ID_keep_drugbank.drop('source_database', 1)
drugbank = drugbank_to_genes_ID_keep_drugbank['source']
#drugcentral = drugcentral['source']

#mixed_df = drugbank.append(drugcentral)

In [13]:
mixed_df = pd.DataFrame(drugbank)

In [14]:
## Drug dataframe prepration for calculating score of a pathway including all of its involving genes 

# Group the targeting genes based on the drugs
mixed_df_groupby_drug = mixed_df.groupby('source')

# Forming a list of unique drugs used further for preparation of dataframe containing drugs and its targeted pathway 
## and all targeted genes involved in that pathway
unique_drug = mixed_df["source"].unique()

In [15]:
## Load the trained classifier

trained_model = joblib.load(open(TRAINED_MODEL, "rb"))

/home/skhatami/miniconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/skhatami/miniconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [16]:
score_list = [-1,1]

synthetic_data_frame = {}


for i in tqdm(range(100)):
    synthetic_drug_gene_score = pd.DataFrame(columns = ["drug",'Gene','relation'])
    num_data_frame = str(i)
    random_selected_drug = random.sample(list(unique_drug), len(unique_drug))
    for drug in random_selected_drug:
        #num_gene_select = gene_per_drug[drug]
        random_selected_gene = random.sample(list(all_hgnc_genes["hgnc_symbol"]), 1)
        #for gene in random_selected_gene:
        random_selected_relation = random.sample(score_list, 1)
        synthetic_drug_gene_score = synthetic_drug_gene_score.append({'drug': drug, 'Gene':random_selected_gene[0], 'relation':random_selected_relation[0]},ignore_index=True)
    synthetic_data_frame[num_data_frame] = synthetic_drug_gene_score
    del synthetic_drug_gene_score

100%|██████████| 100/100 [09:32<00:00,  5.37s/it]


In [17]:
synthetic_pathway_to_score_data_frame = {}

for df_num, df in tqdm(synthetic_data_frame.items()):
    
    pathway_to_score = pd.DataFrame(columns=['drug_ID','pathway','affection_rate','gene_name'])
    
    temp_df = df.groupby('drug')
    df_drug_unique = df["drug"].unique()
    
    for drug in df_drug_unique:
        
    # get the subset of drugbank dataset with regards to the a data
        temp_drug_gene_relation_df = temp_df.get_group(drug)
    
    # drop the drug column to turn it to dict for efficient looping
        temp_drug_gene_relation_df = temp_drug_gene_relation_df.drop("drug",1)
    
    # convert the subset dataframe to dictionary
        temp_gene_score_dict = dict(temp_drug_gene_relation_df.values.tolist())
    
    # loop over pathway_genes_dict genes and pathways
        for pathways, genes in pathway_genes_dict.items():
            temp_gene= genes
            temp_pathway = pathways
        
        # loop over subset dataframe converted dict genes and scores
            for gene, score in temp_gene_score_dict.items():
                gene_temp = gene
                score_temp = score
            
            # find all genes of a pathway and makeing a dataframe out of that with all details (drug,gene,pathway,affecting score)
                if gene_temp in temp_gene:
                    #pathway_to_score.append([drug,temp_pathway,score_temp,gene_temp])
                    pathway_to_score = pathway_to_score.append({'drug_ID':drug,'pathway': temp_pathway, 'affection_rate': score_temp, 'gene_name': gene_temp},ignore_index=True)
    synthetic_pathway_to_score_data_frame[df_num] = pathway_to_score
    del pathway_to_score

100%|██████████| 100/100 [05:15<00:00,  2.96s/it]


In [18]:
synthetic_pathway_scores_data_frame = {}

for df_num, df in tqdm(synthetic_pathway_to_score_data_frame.items()):
    
    pathway_scores = pd.DataFrame(columns=['drug_ID','Pathway', 'Finall_affected_score'])
    
    pathway_to_score_groupby = df.groupby(['drug_ID','pathway'])
    
    for drug,path,score,gene in df.values:
    
        # get the subset of last step prepared dataframe with regards to the drug and correponding pathway (considering all of its involving genes)
        temp_pathway_to_score_df = pathway_to_score_groupby.get_group((drug,path))
    
        # calculating the sum of the scores for all the genes of a pathway
        temp_affected_score = temp_pathway_to_score_df['affection_rate'].sum()
    
        # calculating the mean 
        finall_affected_score = temp_affected_score / (temp_pathway_to_score_df.shape[0])
    
        # make a dataframe dataframe with a score per drug per pathway
        pathway_scores = pathway_scores.append({'drug_ID':drug,'Pathway': path, 'Finall_affected_score': finall_affected_score},ignore_index=True)
    
    synthetic_pathway_scores_data_frame[df_num] = pathway_scores
    del pathway_scores

100%|██████████| 100/100 [02:17<00:00,  1.25s/it]


In [19]:
## Splite samples based on our desired lables

def splite_samples(raw_data, desired_label):
    
    # Split the subset of pateints having desired lable
    desired_label_sample= raw_data.loc[raw_data['label'] == desired_label]
    
    # Dataframe including the other subset of patients with undisred lable
    undesired_label_sample = pd.concat([raw_data, desired_label_sample]).drop_duplicates(keep=False)
    
    return desired_label_sample, undesired_label_sample

In [20]:
desired_label_sample, undesired_label_sample = splite_samples(raw_data,1)

desired_label_sample.drop('label', axis=1, inplace=True)
undesired_label_sample.drop('label', axis=1, inplace=True)

patients_mean_pathway = {}
healthy_mean_pathway = {}

for pathway in desired_label_sample:
    mean = desired_label_sample[pathway].mean()
    patients_mean_pathway[pathway] = mean

mean = 0
    
for pathway in undesired_label_sample:
    mean = undesired_label_sample[pathway].mean()
    healthy_mean_pathway[pathway] = mean
    
    
patients_mean_pathway_df = pd.DataFrame(patients_mean_pathway.items(), columns=['pathway', 'mean_patient'])
healthy_mean_pathway_df = pd.DataFrame(healthy_mean_pathway.items(), columns=['pathway', 'mean_healthy'])

mean_patient_healthy_pathway = pd.merge(healthy_mean_pathway_df, patients_mean_pathway_df, on=["pathway"])
#mean_patient_healthy_pathway = pd.merge(mean_patient_healthy_pathway, path_coef, on=["pathway"])

In [21]:
mean_patient_healthy_pathway["diff_mean"] = abs(mean_patient_healthy_pathway["mean_healthy"] - mean_patient_healthy_pathway["mean_patient"])
mean_patient_healthy_pathway = mean_patient_healthy_pathway.sort_values(by = 'diff_mean',ascending = False)

In [22]:
## Modify the pathway score of each patient with regards to each drug available in drugbank

def path_score_modification(drug_name, raw_data, desired_label,mean_patient_healthy_pathway):
     
    desired_path_score_changed_sample, undesired_path_score_changed_sample = splite_samples(raw_data, desired_label)
    
    # Get subset of dataframe with a score per drug per pathway with regards to selected drug
    temp_pathway_drug_all_gene_score = df.groupby('drug_ID')
    pathway_drug_including_all_gene_score = temp_pathway_drug_all_gene_score.get_group(drug_name)
    
    # Dictionary of pathways affected by the drug to their respective scores
    affected_pathway_to_score = {
        pathway: score
        for _, pathway, score in pathway_drug_including_all_gene_score.values
    }
                 
    # For each sample id
    for sample in range(len(desired_path_score_changed_sample)):
            
        # For each pathway that we have to modify a score in all patients since it is targetted by the drug
        for pathway in affected_pathway_to_score:
                        
            # Get related affection scored calculated per drug per pathway
            affection_score = affected_pathway_to_score[pathway]
            
            if pathway in desired_path_score_changed_sample.columns:
                
                pathway_column = desired_path_score_changed_sample.columns.get_loc(pathway)
                current_score = desired_path_score_changed_sample.iloc[sample, pathway_column]
                temp_mean_patient_healthy_pathway = mean_patient_healthy_pathway.loc[mean_patient_healthy_pathway['pathway'] == pathway]


                
                if affection_score > 0:
                    
                    if temp_mean_patient_healthy_pathway.iloc[0,3] > np.quantile(mean_patient_healthy_pathway['diff_mean'], 0.75):
                    
                        desired_path_score_changed_sample.iloc[sample, pathway_column] = 20 * abs(current_score)
                        
                    elif np.quantile(mean_patient_healthy_pathway['diff_mean'], 0.75) >= temp_mean_patient_healthy_pathway.iloc[0,3] >= np.quantile(mean_patient_healthy_pathway['diff_mean'], 0.5):
                        
                        desired_path_score_changed_sample.iloc[sample, pathway_column] = 5 * abs(current_score)
                    
                    else:
                        desired_path_score_changed_sample.iloc[sample, pathway_column] = 10 * abs(current_score)
                        
 
                elif affection_score == 0:
                    
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = current_score

                    
                else:
                    
                    if temp_mean_patient_healthy_pathway.iloc[0,3] > np.quantile(mean_patient_healthy_pathway['diff_mean'], 0.75):
                    
                        desired_path_score_changed_sample.iloc[sample, pathway_column] = -20 * abs(current_score)
                        
                    elif np.quantile(mean_patient_healthy_pathway['diff_mean'], 0.75) >= temp_mean_patient_healthy_pathway.iloc[0,3] >= np.quantile(mean_patient_healthy_pathway['diff_mean'], 0.5):
                        
                        desired_path_score_changed_sample.iloc[sample, pathway_column] = -5 * abs(current_score)
                    
                    else:
                        desired_path_score_changed_sample.iloc[sample, pathway_column] = -10 * abs(current_score)
    
    return desired_path_score_changed_sample

In [23]:
def auc_per_drug(drug_data_set,model,data,desired_lable, raw_data_set_lable,mean_patient_healthy_pathway):
    
    pathway_drug_score_prad = pd.DataFrame(columns=['drug','label_changed_ratio'])

    for drug in df["drug_ID"].unique():
        ratio = 0
        temp_data_set = path_score_modification(drug,raw_data,desired_lable,mean_patient_healthy_pathway)
        prepared_data_set_for_prediction = temp_data_set.iloc[:,:311]
        prediction = trained_model.predict(prepared_data_set_for_prediction)
        for l_o,l_p in zip(raw_data.label,prediction):
            if l_o != l_p and l_o == 1:
                ratio = ratio + 1
        ratio = (ratio/lable_list.count(1))
        pathway_drug_score_prad = pathway_drug_score_prad.append({'drug': drug,'label_changed_ratio': ratio},ignore_index=True)
        
    return pathway_drug_score_prad       

In [24]:
permut_dist = pd.DataFrame(columns=['DataFrame_num', 'Prioritized_drugs_num'])


for num_df,df in tqdm(synthetic_pathway_scores_data_frame.items()):
    number_of_prioritized_drugs = 0
        
    pathway_drug_score_lihc_df = auc_per_drug(df,trained_model,raw_data,1,raw_data.label,mean_patient_healthy_pathway)
        
    for drug,label_changed_ratio in pathway_drug_score_lihc_df.values:
        if label_changed_ratio >=0.8:
            number_of_prioritized_drugs = number_of_prioritized_drugs + 1
    print(num_df,number_of_prioritized_drugs)
                
    permut_dist = permut_dist.append({'DataFrame_num': num_df, 'Prioritized_drugs_num': number_of_prioritized_drugs},ignore_index=True)

  1%|          | 1/100 [08:54<14:42:24, 534.79s/it]

0 13


  2%|▏         | 2/100 [21:11<16:12:39, 595.51s/it]

1 19


  3%|▎         | 3/100 [32:34<16:45:01, 621.67s/it]

...

97 14


 99%|█████████▉| 99/100 [15:43:45<08:33, 513.53s/it]

98 13


100%|██████████| 100/100 [15:52:06<00:00, 509.86s/it]

99 14


In [25]:
permut_dist.to_csv('/home/skhatami/Projects/drug-pathway-revert/drug-pathways-revert/ssGSEA_Results/permut_dist_prad.txt', sep = '\t', index = False)